In [1]:
import os
import itk
import SimpleITK as sitk

In [2]:
os.getcwd()

'/home/shwetank/Inter IIT'

In [3]:
import sys

In [4]:
sys.path.append('/home/shwetank/vtkBuild/lib/python3.5/site-packages')

In [5]:
import vtk

In [6]:
sys.path.append('/home/shwetank/vtkBuild/lib/python3.5/site-packages')

In [7]:
from vtk.util import numpy_support 
import os
import numpy
from matplotlib import pyplot, cm
import plotly.plotly as py
import plotly.graph_objs as go

In [8]:
PathDicom = "./Sampdata/SE5/"
reader  = vtk.vtkDICOMImageReader()
reader.SetDirectoryName(PathDicom)
reader.Update()

In [9]:
_extent = reader.GetDataExtent()
ConstPixelDims = [_extent[1]-_extent[0]+1,_extent[3]-_extent[2]+1, _extent[5]-_extent[4]+1]

In [10]:
ConstPixelSpacing = reader.GetPixelSpacing()

In [11]:
print(ConstPixelDims)
print(ConstPixelSpacing)

[512, 512, 187]
(0.4375, 0.4375, 0.9999923706054688)


In [12]:
#shiftScale = vtk.vtkImageShiftScale()
#shiftScale.SetScale(reader.GetRescaleSlope())
#shiftScale.SetShift(reader.GetRescaleOffset())
#shiftScale.SetInputConnection(reader.GetOutputPort())
#shiftScale.Update()

In [13]:
print(reader.GetRescaleSlope())
print(reader.GetRescaleOffset())

1.0
-1024.0


In [14]:
# Get the 'vtkImageData' object from the reader
imageData = reader.GetOutput()
# Get the 'vtkPointData' object from the 'vtkImageData' object
pointData = imageData.GetPointData()
# Ensure that only one array exists within the 'vtkPointData' object
assert (pointData.GetNumberOfArrays()==1)
# Get the `vtkArray` (or whatever derived type) which is needed for the `numpy_support.vtk_to_numpy` function
arrayData = pointData.GetArray(0)
# Convert the `vtkArray` to a NumPy array
ArrayDicom = numpy_support.vtk_to_numpy(arrayData)
# Reshape the NumPy array to 3D using 'ConstPixelDims' as a 'shape'
ArrayDicom = ArrayDicom.reshape(ConstPixelDims, order='F')

In [15]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot,iplot

In [16]:
init_notebook_mode(connected=True)

In [17]:
trace = go.Heatmap(z = numpy.rot90(ArrayDicom[188,:,:]))
data = [trace]
iplot(data, filename = 'basic-heatmap')

In [18]:
threshold = vtk.vtkImageThreshold()
threshold.SetInputConnection(reader.GetOutputPort())
threshold.ThresholdBetween(200,500)  # remove all soft tissue
threshold.ReplaceInOn()
threshold.SetInValue(1)  # set all values below 400 to 0
threshold.ReplaceOutOn()
threshold.SetOutValue(0)  # set all values above 400 to 1
threshold.Update()

In [19]:
dmc = vtk.vtkDiscreteMarchingCubes()
dmc.SetInputConnection(threshold.GetOutputPort())
dmc.GenerateValues(1,1,1)
dmc.Update()

In [20]:
mapper = vtk.vtkPolyDataMapper()
mapper.SetInputConnection(dmc.GetOutputPort())

In [21]:
actor = vtk.vtkActor()

In [22]:
actor.SetMapper(mapper)

In [23]:
renderer = vtk.vtkRenderer()

In [24]:
renderer.AddActor(actor)

In [25]:
renderer.SetBackground(1.0,1.0,1.0)

In [26]:
camera = renderer.MakeCamera()

In [27]:
camera.SetPosition(-200.0,100.5,122.0)

In [28]:
camera.SetFocalPoint(301.0, 245.5, 122.0)
camera.SetViewAngle(30.0)
camera.SetRoll(0.0)

In [29]:
renderer.SetActiveCamera(camera)

In [30]:
import vtk
from IPython.display import Image
def vtk_show(renderer, width=400, height=300):
    """
    Takes vtkRenderer instance and returns an IPython Image with the rendering.
    """
    renderWindow = vtk.vtkRenderWindow()
    renderWindow.SetOffScreenRendering(1)
    renderWindow.AddRenderer(renderer)
    renderWindow.SetSize(width, height)
    renderWindow.Render()
     
    windowToImageFilter = vtk.vtkWindowToImageFilter()
    windowToImageFilter.SetInput(renderWindow)
    windowToImageFilter.Update()
     
    writer = vtk.vtkPNGWriter()
    writer.SetWriteToMemory(1)
    writer.SetInputConnection(windowToImageFilter.GetOutputPort())
    writer.Write()
    data = str(buffer(writer.GetResult()))
    
    return Image(data)


In [32]:
renderWindow = vtk.vtkRenderWindow()
renderWindow.AddRenderer(renderer)
renderWindowInteractor = vtk.vtkRenderWindowInteractor()
renderWindowInteractor.SetRenderWindow(renderWindow)
renderWindow.SetSize(600,600)
renderWindow.Render()
renderWindowInteractor.Start()
#vtk_show(renderer, 600, 600)

In [37]:
writer = vtk.vtkSTLWriter()
writer.SetInputConnection(dmc.GetOutputPort())
writer.SetFileTypeToBinary()
writer.SetFileName("fiducial.stl")
writer.Write()

1

In [ ]:
# Get the 'vtkImageData' object from the reader
imageData = threshold.GetOutput()
# Get the 'vtkPointData' object from the 'vtkImageData' object
pointData = imageData.GetPointData()
# Ensure that only one array exists within the 'vtkPointData' object
assert (pointData.GetNumberOfArrays()==1)
# Get the `vtkArray` (or whatever derived type) which is needed for the `numpy_support.vtk_to_numpy` function
arrayData = pointData.GetArray(0)
# Convert the `vtkArray` to a NumPy array
ArrayDcm = numpy_support.vtk_to_numpy(arrayData)
# Reshape the NumPy array to 3D using 'ConstPixelDims' as a 'shape'
ArrayDcm = ArrayDcm.reshape(ConstPixelDims, order='F')

In [ ]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot,iplot
init_notebook_mode(connected=True)
trace = go.Heatmap(z = numpy.rot90(ArrayDcm[188,:,:]))
data = [trace]
iplot(data, filename = 'basic-heatmap')

In [34]:
import itk

In [35]:
Dimension = 3
PixelType = itk.ctype('signed short')
ImageType = itk.Image[PixelType, Dimension]

In [36]:
namesGen = itk.GDCMSeriesFileNames.New()
namesGen.SetUseSeriesDetails(True)
namesGen.SetGlobalWarningDisplay(False)
namesGen.SetDirectory("./Sampdata/SE5/")

In [37]:
seriesUID = namesGen.GetSeriesUIDs()

if len(seriesUID) < 1:
    print('No DICOMs in: ' + dirName)
    sys.exit(1)

print('The directory: ' + "./Sampdata/SE5")
print('Contains the following DICOM Series: ')
for uid in seriesUID:
    print(uid)


The directory: ./Sampdata/SE5
Contains the following DICOM Series: 
1.2.840.113619.2.80.3994079523.17520.1467023411.1.4.1.4500.624997139512512
1.2.840.113619.2.80.3994079523.17520.1467023411.1.4.1.4501512512


In [38]:
seriesFound = False
for uid in seriesUID:
    seriesIdentifier = uid
    if len(sys.argv) > 3:
        seriesIdentifier = sys.argv[3]
        seriesFound = True
    print('Reading: ' + seriesIdentifier)
    fileNames = namesGen.GetFileNames(seriesIdentifier)

    reader = itk.ImageSeriesReader[ImageType].New()
    dicomIO = itk.GDCMImageIO.New()
    reader.SetImageIO(dicomIO)
    reader.SetFileNames(fileNames)
    
    if seriesFound:
        break

Reading: 1.2.840.113619.2.80.3994079523.17520.1467023411.1.4.1.4500.624997139512512
Reading: 1.2.840.113619.2.80.3994079523.17520.1467023411.1.4.1.4501512512


In [39]:
thresholdFilter = itk.BinaryThresholdImageFilter[ImageType,ImageType].New()
thresholdFilter.SetInput(reader.GetOutput())

In [40]:
thresholdFilter.SetLowerThreshold(200)
thresholdFilter.SetUpperThreshold(600)
thresholdFilter.SetOutsideValue(0)
thresholdFilter.SetInsideValue(1)
threshold.update()

In [1]:
import itk

In [2]:
itkFilter = itk.UC

In [28]:
k = reader.GetImageOrientationPatient()

In [29]:
k

(1.0, 0.0, 0.0, 0.0, 1.0, 0.0)